In [ ]:
import gmaps
import pandas as pd
import requests as re
import json


In [ ]:
from config import gkey

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords,
    "rankby": "distance",
    "type": "hospital",
    "key": gkey,
}